## Hyperparameter Tuning

In our pursuit of optimizing predictive performance for California housing price prediction, we turn our attention towards hyperparameter tuning. 

Hyperparameters play a pivotal role in shaping the behavior and performance of machine learning models, and fine-tuning them can lead to significant improvements in predictive accuracy and generalization.

#### Loading and preparing the data

In [34]:
from sklearn.datasets import  fetch_california_housing
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
california = fetch_california_housing()
print(california["DESCR"])

In [ ]:
df_cali = pd.DataFrame(california["data"], columns = california["feature_names"])
df_cali["median_house_value"] = california["target"]

df_cali.head()

#### Normalization & Feature Selection

Like we did in Feature Engineering lesson, we are going to normalize our data and select a subset of columns as our features.

#### Train Test Split

In [37]:
features = df_cali.drop(columns = ["median_house_value","AveOccup", "Population", "AveBedrms"])
target = df_cali["median_house_value"]

In [38]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

Create an instance of the normalizer

In [ ]:
normalizer = MinMaxScaler()

normalizer.fit(X_train)

In [40]:
X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)

In [41]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)

# Grid Search

**Grid Search** - we define a grid of hyperparameter values we want to try. Grid Search tries all possible combinations.

So far, our best model was AdaBoost yield a R-Squared of 0.83.


Let's see how we fine tune our model, in order to that, we will optimize the following hyperparameters:

- **n_estimators:** number of estimators, in this case, number of trees

- **max_leaf_nodes:** maxium number of total leafs to consider

- **max_depth:** maxium number of levels in each tree

- First we define the grid with values to consider when train all possible combinations.

In [42]:
grid = {"n_estimators": [50, 100, 200,500],
        "estimator__max_leaf_nodes": [250, 500, 1000, None],
        "estimator__max_depth":[10,30,50]}

In [43]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor())

In [44]:
model = GridSearchCV(estimator = ada_reg, param_grid = grid, cv=5, n_jobs=-1, verbose=2)

In [ ]:
model.fit(X_train_norm, y_train)

- After training, we check what are the best values for the hyperparameters that we have tested.

In [ ]:
model.best_params_

- You can retrieve the best model with the best parameters when accessing **best_estimator_** attribute

In [47]:
best_model = model.best_estimator_

- Evaluate our model

In [ ]:
pred = best_model.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", best_model.score(X_test_norm, y_test))

# Random Search

**Random Search** - we define probability distributions for each hyperparameter, from which random values are sampled. It’s up to the researcher to set the maximum number of combinations.

In [49]:
grid = {"n_estimators": [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
        "estimator__max_leaf_nodes": [int(x) for x in np.linspace(start = 500, stop = 3000, num = 10)],
        "estimator__max_depth":[int(x) for x in np.linspace(10, 110, num = 11)]}

In [50]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor())

model = RandomizedSearchCV(estimator = ada_reg, param_distributions = grid, n_iter = 10, cv = 5, n_jobs = -1)

In [ ]:
model.fit(X_train_norm,y_train)

In [ ]:
model.best_params_

- You can retrieve the best model with the best parameters when accessing **best_estimator_** attribute

In [53]:
best_model = model.best_estimator_

- Evaluate our model

In [ ]:
pred = best_model.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", best_model.score(X_test_norm, y_test))

We dont guarantee these hyperparameters are optimal! We can just guarantee that these are the best from the ones we tried!